In [2]:
import os
import configparser
from linebot import  LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, LocationMessage,TextSendMessage, ImageSendMessage, StickerSendMessage, LocationSendMessage, QuickReply, QuickReplyButton, MessageAction
import random
import googlemaps
configFilename = 'APIKeys.ini'
if not os.path.isfile(configFilename):
    with open(configFilename, 'a') as f:
        f.writelines(["[Default]\n", "Google_API_KEY= your Google API KEY\n", "linebot_api_key= your LineBot API key\n", "handler_key= your WebhookHandler key"])
        print('input your API keys in APIKeys.ini')
        exit()
# get account info fomr ini config file
config = configparser.ConfigParser()
config.read(configFilename)
Google_API_KEY = config['Default']['Google_API_KEY']
linebot_api_key = config['Default']['linebot_api_key']
handler_key = config['Default']['handler_key']


gmaps = googlemaps.Client(Google_API_KEY)
linebot = LineBotApi(linebot_api_key)
handler = WebhookHandler(handler_key)

In [3]:
from flask import Flask
app = Flask(__name__)
from flask import Flask, request, abort

# #get url with only lat and lng
# def geturl(lat, lng):
#     query_result = gmaps.find_place(input='restaurant', input_type='textquery', fields=['place_id'], location_bias='circle:3000@{},{}'.format(lat, lng))

#     # Get the place ID of the closest match
#     place_id = query_result['candidates'][0]['place_id']

#     # Look up the place details and get the URL
#     place_details = gmaps.place(place_id=place_id, fields=['url'])
#     url = place_details['result']['url']
#     return url
#get one url with keyword
def geturl(lat, lng, keyword=''):
    places = gmaps.places_nearby(location=(lat, lng), radius=3000, type='restaurant', keyword=keyword)

    place_details = gmaps.place(place_id=['place_id'], fields=['url'])
    url = place_details['result']['url']

    return url
#get multiple urls with keyword
def geturls(lat, lng, keyword = '', maxcount = 3):

    places = gmaps.places_nearby(location=(lat, lng), radius=3000, type='restaurant', keyword=keyword)
    #create a list of urls
    urls = []
    count = 0
    for place in places['results']:
        place_details = gmaps.place(place_id=place['place_id'], fields=['url'])
        urls.append(place_details['result']['url'])
        count += 1
        if count >= maxcount:
            break
    return urls


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    #print(body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'
#@handler.add(MessageEvent, message=TextMessage)
@handler.add(MessageEvent)
#@handler.add(MessageEvent,message=LocationMessage)
def handle_message(event):
    #print(event.message.type, end='\n')#debuggin use
    eatflag = False
    keyword = ''
    if event.message.type == 'location' and eatflag == True:
        try:
            latitude = event.message.latitude
            longitude = event.message.longitude
            message = TextSendMessage(
                text=str(geturl(lat=latitude, lng=longitude, keyword=keyword))
            )
            linebot.reply_message(event.reply_token, message)
            eatflag = False
        except:
            linebot.reply_message(event.reply_token, TextSendMessage(text = "發生錯誤！"))
    elif event.message.text == '吃什麼':
        try:
            message = TextSendMessage(
                text='請選擇菜式或自行輸入',
                quick_reply = QuickReply(
                    items=[
                        QuickReplyButton(
                            action=MessageAction(label="中式",text="中式")
                        ),
                        QuickReplyButton(
                            action=MessageAction(label="西式",text="西式")
                        ),
                        QuickReplyButton(
                            action=MessageAction(label="日式",text="日式")
                        ),
                        QuickReplyButton(
                            action=MessageAction(label="速食",text="速食")
                        ),
                        QuickReplyButton(
                            action=MessageAction(label="小吃",text="小吃")
                        ),
                    ]
                )
            )
            linebot.reply_message(event.reply_token, message)
            eatflag = True
        except:
            linebot.reply_message(event.reply_token, TextSendMessage(text = "發生錯誤！"))
    elif eatflag == True and event.message.type == 'text' :
        try:
            message = TextSendMessage(
                text = '請輸入地址'
            )
            keyword = event.message.text
            linebot.reply_message(event.reply_token, message)
        except:
            linebot.reply_message(event.reply_token, TextSendMessage(text = "發生錯誤！"))
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2023 02:10:04] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2023 02:10:21] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2023 02:10:23] "POST /callback HTTP/1.1" 200 -
[2023-01-07 02:11:07,024] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "c:\Users\user\Documents\GitHub\111-Fall\linebot\.venv\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\user\Documents\GitHub\111-Fall\linebot\.venv\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\user\Documents\GitHub\111-Fall\linebot\.venv\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\user\Documents\GitHub\111-Fall\linebot\.venv\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    retur

In [ ]:
# lat = 24.971148
# lng = 121.263871
# keyword = '速食'
# places = gmaps.places_nearby(location=(lat, lng), radius=3000, type='restaurant', keyword=keyword)
# #places = gmaps.places_nearby(location=(lat, lng), rankby='distance', type='restaurant', keyword=keyword)
# cnt = 0
# for place in places['results']:
#     place_details = gmaps.place(place_id=place['place_id'], fields=['url'])
#     url = place_details['result']['url']
#     print(url)
#     cnt = cnt + 1
#     if cnt >= 3:
#         break
# # place_details = gmaps.place(place_id=places['place_id'], fields=['url'])
# # url = place_details['result']['url']
